**Importy**

In [2]:
import regex
from csv import writer
import pandas as pd 
import numpy as np
from stop_words import get_stop_words

In [3]:
import majka
from majka import Majka
morph = Majka('../data/w-lt.sk.fsa')

morph.flags |= majka.ADD_DIACRITICS  # find word forms with diacritics
morph.flags |= majka.DISALLOW_LOWERCASE  # do not enable to find lowercase variants
morph.flags |= majka.IGNORE_CASE  # ignore the word case whatsoever
morph.flags = 0  # unset all flags

morph.tags = False  # return just the lemma, do not process the tags
# morph.tags = True  # turn tag processing back on (default)

morph.first_only = True  # return only the first entry
# morph.first_only = False  # return all entries (default)

In [4]:
from corpy.morphodita import Tagger
tagger = Tagger("../data/slovak-morfflex-pdt-170914.tagger")

INFO:corpy.morphodita:Loading tagger.


In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rusna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rusna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Premenne**

In [6]:
debug_filepath = "../data/test_data_1page.xml"
input_filepath = "../data/skwiki-latest-pages-articles.xml"
output_filepath = "../data/anchors.csv"

**Funkcie**

In [7]:
def open_read(input_filepath,output_filepath,process):
    with open(input_filepath, encoding='UTF-8') as input_file ,open(output_filepath, 'w+', encoding='UTF-8' ) as output_file:
        csv_writer = writer(output_file, delimiter='|')
        csv_writer.writerow(["Link", "Alt_Text"])
        for line in input_file:
            process(line,csv_writer)

In [8]:
def process(line,csv_writer):
    regex_patern = '\[\[([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)\|?([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)?\]\]'
    matches = regex.findall(regex_patern, line)
    for match in matches:
        list_matche = list(filter(None, match))
        
#         print(list_matche)
        
        csv_writer.writerow(list_matche)

In [9]:
def replace(text, regex_pattern, substitution):
    if not isinstance(text, str):
        return text
    return regex.sub(regex_pattern,substitution,text)

In [10]:
def tokenizer_modded(text):
    sentences = nltk.sent_tokenize(text)
    tokens = []
    for sent in sentences:
        tokens.append(nltk.word_tokenize(sent))
    flatten_list = [j for sub in tokens for j in sub] 
    result = [token.lower() for token in flatten_list if token not in ".,?!...-"]
    
    return result

In [11]:
def tag(word):
    result=[]
    tokens = list(tagger.tag(word))
#     for i in range(len(tokens)):
#         if any(s in tokens[i].tag for s in need):
#             result.append(tokens[i].word)
            
    return tokens

In [12]:
def lematizer_modded(text):
#     stop_words = get_stop_words('czech')
#     stop_words2 = ['a','s',' s',' a',' v','1','2','3','4','5','6','7','8','9','0']
#     stop_words = stop_words + stop_words2
    result = []
    for word in text:
        morph_word = morph.find(word)
        if morph_word:
            result.append(morph_word[0]['lemma'])
        else:
            result.append(word)
#     for word in result:  # iterating on a copy since removing will mess things up
#         if word in stop_words:
#             result.remove(word)
    return result

**Priebeh spracovania**

In [13]:
open_read(debug_filepath,output_filepath,process)

In [14]:
for index,chunk in enumerate(pd.read_csv(output_filepath, chunksize=15, delimiter='|')):
    chunk['Link'] = chunk['Link'] \
    .apply(lambda x: x if x is not np.NaN else None) \
    .apply(replace,regex_pattern = r"[\\\/:#.\_-]|({{.*}})|\d*px|\d",substitution =' ')\
    .apply(replace,regex_pattern = r"([ ]+)",substitution =' ')\
    .map(lambda text: lematizer_modded(tokenizer_modded(text)))
    
    chunk['Alt_Text'] = chunk['Alt_Text'] \
    .apply(lambda x: x if x is not np.NaN else None) \
    .apply(replace,regex_pattern = r"([ ]+)",substitution =' ')\
    .apply(replace,regex_pattern = r"[\\\/:#.\_-]|({{.*}})|\d*px|\d",substitution =' ')\
    .map(lambda text: lematizer_modded(tokenizer_modded(text)) if text is not None else None )
    
    
    print('chunk ------------------------------------------------------------------------------',index)    
    print(chunk)
    chunk.to_csv('../data/done.csv', mode = 'a+', index=False, encoding='UTF-8')


chunk ------------------------------------------------------------------------------ 0
                                                 Link         Alt_Text
0               [šablóna, hlavná, stránka, aktualita]             None
1   [special, permalink, preview, hlavná, stránka,...          [návrh]
2                           [wikipédia, o, wikipédii]   [o, wikipédii]
3                               [wikipédia, príručka]       [príručka]
4                                      [pomoc, obsah]          [pomoc]
5                  [wikipédia, zoznam, dobrý, článok]  [dobrý, článok]
6                            [kategória, wikiportály]         [portál]
7                    [kategória, základný, kategória]      [kategória]
8                          [wikipédia, rýchly, index]    [a, z, index]
9                                [súbor, bluebg, png]          [link=]
10                                                 []             None
11                               [súbor, bluebg, png]        

chunk ------------------------------------------------------------------------------ 9
                             Link               Alt_Text
135  [image, cd, tableimage, png]  [cd, tableimage, png]
136                  [user, beno]                 [beno]
137  [image, ca, tableimage, png]  [ca, tableimage, png]
138                  [user, beno]                 [beno]
139   [image, b, tableimage, png]   [b, tableimage, png]
140                  [user, beno]                 [beno]
141  [image, br, tableimage, png]  [br, tableimage, png]
142                  [user, beno]                 [beno]
143  [image, be, tableimage, png]  [be, tableimage, png]
144                  [user, beno]                 [beno]
145  [image, as, tableimage, png]  [as, tableimage, png]
146                  [user, beno]                 [beno]
147  [image, ar, tableimage, png]  [ar, tableimage, png]
148                  [user, beno]                 [beno]
149  [image, al, tableimage, png]  [al, tableimage, png]
c